### Querying current flight positions from OpenSky

In [ ]:
from urllib import request

with request.urlopen("https://opensky-network.org/api/states/all") as response:
    print(response.read().decode("utf-8"))

### Filtering flight positions from OpenSky

In [ ]:
import json
from urllib import request

with request.urlopen("https://opensky-network.org/api/states/all?lamin=45.8389&lomin=5.9962&lamax=47.8229&lomax=10.5226") as response:
    print(json.loads(response.read().decode("utf-8")))

### Creating events from flight data

In [ ]:
def response_to_events(api_response):
    flight_events = []
    for update in api_response["states"]:
        flight_events.append(
            {
                "icao24": update[0],
                "origin_country": update[2],
                "time_position": update[3],
                "longitude": update[5],
                "latitude": update[6],
                "velocity": update[9],
                "true_track": update[10],
            }
        )
    return sorted(flight_events, key=lambda x: x["time_position"])

### A flight update event generator

In [ ]:
import time
from urllib import request

def get_events(
    lat_min=45.8389,
    long_min=5.9962,
    lat_max=47.8229,
    long_max=10.5226,
    interval_sec=10,
):
    url = f"https://opensky-network.org/api/states/all?lamin={lat_min}&lomin={long_min}&lamax={lat_max}&lomax={long_max}"
    for _ in range(3):
        with request.urlopen(url) as response:
            yield from response_to_events(json.loads(response.read().decode("utf-8")))
        time.sleep(interval_sec)

### Calling the event generator

In [ ]:
for event in get_events():
    print(event)